# Job Market for Data Professionals in NL - Web Scraping

In [1]:
# Import Libraries 
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re

In [2]:
# Search keywords Data Engineer, Data Analyst, Data Scientist, Business Intelligence
url = 'https://www.indeed.nl/jobs?q=data+analist%2C+data+scientist%2C+data+engineer%2C+business+intelligence&l=Nederland&sort=date'

In [3]:
driver = webdriver.Chrome("./chromedriver/chromedriver.exe")

In [4]:
driver.get(url)
driver.implicitly_wait(4)

In [5]:
jobs = pd.DataFrame(columns=["Title", "Location", "Company", "Salary", "Since", "Description", 'Company Url'])



for i in range(0, 1000, 10):
    
    driver.get(url+'&start='+str(i))
    driver.implicitly_wait(4)
    
    all_jobs = driver.find_elements_by_class_name('result')
    
    for job in all_jobs:
        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser')

        try:
            title = soup.find("a", class_="jobtitle").text.replace('\n', '')
        except:
            title = 'N/A'

        try:
            company = soup.find(class_="company").text.replace('\n', '').strip()
        except:
            company = 'N/A'
     

        try: 
            location = soup.find(class_="location accessible-contrast-color-location").text
        except:
            location = 'N/A'

        try:
            salary = soup.find(class_="salary").text.replace("\n", "").strip()
        except:
            salary = 'N/A'
        
        try:
            posted = soup.find(class_="date").text
        except:
            posted = 'N/A'
            
        articles = soup.find_all('div', class_='sjcl')
        try: 
            for article in articles:
                a = article.find('a', class_='turnstileLink')['href']
        except:
            a = 'N/A'
            
     
   
    
            
            
    

#         sum_div = job.find_elements_by_class_name("summary")[0]
#         try:
#             sum_div.click()
#         except:
#             close_button = job.find_elements_by_class_name("icl-Icon icl-Icon--sm  icl-Icon--black close")[0]
#             close_button.click()
#             sum_div.click()

#         job_desc = job.find_element_by_id('vjs-desc').text



        jobs = jobs.append({'Title':title, 'Salary': salary, 'Location': location, 'Company': company, 'Since':posted, 'Company Url': a}, ignore_index=True)

In [6]:
# Get some info from the companies 

company_url = jobs['Company Url']

base_url = 'https://www.indeed.nl'

industries = []
company_size = []

for url in company_url:  
    if url != "N/A":
        html = requests.get(base_url+url).content 
        soup = BeautifulSoup(html, "lxml") 
    
        company_info = soup.find_all('div', class_="cmp-AboutMetadata-itemInner")        
        industry = 'N/A'
        size = 'N/A'

        for info in company_info:
            if re.search('Branche', str(info)):
                industry = info.text.split('Branche')[1]   
            if re.search('Medewerkers', str(info)):
                size = info.text.split('Medewerkers')[1]
        
        industries.append(industry)
        company_size.append(size)
           
    else:
        industries.append("N/A")
        company_size.append("N/A")

In [7]:
jobs['Industry'] = industries
jobs['Company size'] = company_size

In [8]:
# test

# url = 'https://www.indeed.nl/cmp/Tennet'

# html = requests.get(url).content 
# soup = BeautifulSoup(html, "lxml") 
    
# test = soup.find_all('div', class_="cmp-AboutMetadata-itemInner")

# test
# for row in test:
#     if re.search('Branche', str(row)):
#         ind = row.text
#         print(ind)
        

In [9]:
jobs.head()

,Title,Location,Company,Salary,Since,Description,Company Url,Industry,Company size
0,Data Scientist,Den Haag,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
1,Associate Scientist USP,Leiden,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
2,Cloud Engineer AWS,Amsterdam,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
3,Web Analist,Amsterdam Centrum,DPG Media,N/A,Net geplaatst,NaN,/cmp/Dpg-Media,N/A,N/A
4,"Senior Full Stack Software Developer (Python, ...",Utrecht Zuidwest,Zero foodwaste,N/A,Net geplaatst,NaN,N/A,N/A,N/A


In [10]:
jobs.to_csv('jobs_all_base.csv', index=False)

In [11]:
len(jobs)

1500

In [13]:
jobs.head(-60)

,Title,Location,Company,Salary,Since,Description,Company Url,Industry,Company size
0,Data Scientist,Den Haag,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
1,Associate Scientist USP,Leiden,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
2,Cloud Engineer AWS,Amsterdam,HAYS,N/A,Net geplaatst,NaN,/cmp/Hays,Human resources en personeel,5.001 tot 10.000
3,Web Analist,Amsterdam Centrum,DPG Media,N/A,Net geplaatst,NaN,/cmp/Dpg-Media,N/A,N/A
4,"Senior Full Stack Software Developer (Python, ...",Utrecht Zuidwest,Zero foodwaste,N/A,Net geplaatst,NaN,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...
1435,Data Scientist en een passie voor Health,Amersfoort,CareerValue,N/A,Meer dan 30 dagen geleden,NaN,/cmp/Careervalue,N/A,N/A
1436,Data modeling specialist (in dienst bij Yacht),Amsterdam Centrum,Yacht,N/A,Meer dan 30 dagen geleden,NaN,/cmp/Yacht,N/A,N/A
1437,Software Engineer Traineeship in Amsterdam,Amsterdam,CareerValue,N/A,Meer dan 30 dagen geleden,NaN,/cmp/Careervalue,N/A,N/A
1438,Data Engineer rol met AWS en Azure,Breukelen,CareerValue,N/A,Meer dan 30 dagen geleden,NaN,/cmp/Careervalue,N/A,N/A


4